In [1]:
import numpy as np
import pandas as pd
import lightgbm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import math
from sklearn import metrics
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import os
import glob

In [2]:

data_path='/home/madnisal/Documents/ML_Project/datasets/'
training_df = pd.read_csv(data_path+'train.csv', index_col="tripid")

In [3]:
def dist_from_coordinates(lat1, lon1, lat2, lon2):
  R = 6371  # Earth radius in km

  #conversion to radians
  d_lat = np.radians(lat2-lat1)
  d_lon = np.radians(lon2-lon1)

  r_lat1 = np.radians(lat1)
  r_lat2 = np.radians(lat2)

  #haversine formula
  a = np.sin(d_lat/2.) **2 + np.cos(r_lat1) * np.cos(r_lat2) * np.sin(d_lon/2.)**2

  haversine = 2 * R * np.arcsin(np.sqrt(a))

  return haversine

In [4]:
training_df['pickup_time'] = pd.to_datetime(training_df['pickup_time'], format="%m/%d/%Y %H:%M")
training_df['drop_time'] = pd.to_datetime(training_df['drop_time'], format="%m/%d/%Y %H:%M")

In [5]:
training_df = training_df.assign(timeOfDay=pd.cut(training_df.pickup_time.dt.hour,[-1, 8, 20, 24],labels=['dawn','day', 'night']))

In [6]:
training_df.loc[training_df['timeOfDay'] == 'day', 'isNormalCharge'] = 1
training_df.loc[training_df['timeOfDay'] != 'day', 'isNormalCharge'] = 0

In [7]:
durations = []
for index,row in training_df.iterrows():
  provided_duration = row['duration'] #first row of location.lat column here
  if math.isnan(provided_duration) or provided_duration <= 0 :
    time_dif = (row['drop_time'] - row['pickup_time']).seconds
    if(time_dif == 0):
        time_dif = np.nan
    durations.append(time_dif)
  else :  
    durations.append(provided_duration)

training_df.insert(4,"time_dif",durations)

In [8]:
new_column = []                    #empty column for distance
for index,row in training_df.iterrows():
  lat1 = row['pick_lat'] #first row of location.lat column here
  lon1 = row['pick_lon'] #first row of location.long column here
  lat2 = row['drop_lat'] #second row of location.lat column here
  lon2 = row['drop_lon'] #second row of location.long column here
  value = dist_from_coordinates(lat1, lon1, lat2, lon2)  #get the distance
  new_column.append(value)   #append the empty list with distance values

training_df.insert(4,"distance",new_column)

In [9]:
training_df['time_driven'] = training_df['duration']  - training_df['meter_waiting']

In [10]:
chargeperhours = []
for index,row in training_df.iterrows():
    if(row['meter_waiting'] == 0):
        chargeperhour = np.nan
    else:
        chargeperhour = (row['meter_waiting_fare'] / row['meter_waiting'] * 3600)
    chargeperhours.append(chargeperhour)

training_df.insert(4,'charge_per_hour',chargeperhours)


In [11]:
training_df['driving_fare'] = training_df['fare']  - training_df['meter_waiting_fare'] - training_df['additional_fare']

In [12]:
avgspeeds = []
for index,row in training_df.iterrows():
    if(row['time_driven'] == 0):
        avgspeed = np.nan
    else:
        avgspeed = (row['distance'] / row['time_driven'] * 3600)
    avgspeeds.append(avgspeed)

training_df.insert(4,"avg_speed",avgspeeds)



In [13]:
costsperkm = []
for index,row in training_df.iterrows():
    if row['distance'] == 0:
        costperkm = np.nan
            
    else:
        costperkm = (row['driving_fare'] / row['distance'])
    costsperkm.append(costperkm)

training_df.insert(4,"cost_per_km",costsperkm)

In [14]:
training_df = training_df.replace({'label': {'incorrect': 0, 'correct' : 1}})

In [15]:
training_df.columns

Index(['additional_fare', 'duration', 'meter_waiting', 'meter_waiting_fare',
       'cost_per_km', 'avg_speed', 'charge_per_hour', 'distance', 'time_dif',
       'meter_waiting_till_pickup', 'pickup_time', 'drop_time', 'pick_lat',
       'pick_lon', 'drop_lat', 'drop_lon', 'fare', 'label', 'timeOfDay',
       'isNormalCharge', 'time_driven', 'driving_fare'],
      dtype='object')

In [16]:
training_columns = ['meter_waiting','meter_waiting_fare','fare','additional_fare', 'distance','cost_per_km', 'avg_speed',  'time_dif','time_driven', 'charge_per_hour', 'driving_fare', 'isNormalCharge','pick_lat','pick_lon','drop_lat','drop_lon']

In [17]:
target_column = ['label']

In [18]:
training_df = training_df.drop(190167541)

In [19]:
x = training_df[training_columns].values
y = training_df[target_column].values

In [20]:
x, x_test, y, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)

In [21]:
training_df['label'].value_counts()

1    15494
0     1681
Name: label, dtype: int64

<h2>Choosing the best model</h2>

In [71]:
def focal_loss_lgb_eval_error(y_true, y_pred, alpha=.25, gamma=2.):
    a,g = alpha, gamma
    p = 1/(1+np.exp(-y_pred))
    loss = -( a*y_true + (1-a)*(1-y_true) ) * (( 1 - ( y_true*p + (1-y_true)*(1-p)) )**g) * ( y_true*np.log(p)+(1-y_true)*np.log(1-p) )
    return 'focal_loss', np.mean(loss), False

In [72]:
from sklearn.metrics import f1_score
def evaluate_macroF1_lgb(y_true, y_pred):  
    y_hat = np.where(y_pred < 0.5, 0, 1) 
    f1 = f1_score(y_true, y_hat, average='macro')
    return ('macroF1', f1, True) 

In [73]:
gkf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [74]:
model = lightgbm.LGBMClassifier(class_weight={0:1.5,1:1}, learning_rate=0.1)

In [75]:
for training_index, testing_index in gkf.split(X=x, y=y):
    x_train_fold, y_train_fold = x[training_index], y[training_index]
    x_test_fold, y_test_fold = x[testing_index], y[testing_index]
    model.fit(x_train_fold, y_train_fold, eval_set=(x_test_fold,y_test_fold),eval_metric = lambda y_true, y_pred: [evaluate_macroF1_lgb(y_true,y_pred)])

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[1]	valid_0's binary_logloss: 0.28951	valid_0's macroF1: 0.474268
[2]	valid_0's binary_logloss: 0.266245	valid_0's macroF1: 0.474268
[3]	valid_0's binary_logloss: 0.248837	valid_0's macroF1: 0.474268
[4]	valid_0's binary_logloss: 0.235365	valid_0's macroF1: 0.474268
[5]	valid_0's binary_logloss: 0.224279	valid_0's macroF1: 0.691565
[6]	valid_0's binary_logloss: 0.214593	valid_0's macroF1: 0.743383
[7]	valid_0's binary_logloss: 0.206098	valid_0's macroF1: 0.79792
[8]	valid_0's binary_logloss: 0.198465	valid_0's macroF1: 0.810983
[9]	valid_0's binary_logloss: 0.192515	valid_0's macroF1: 0.820103
[10]	valid_0's binary_logloss: 0.186296	valid_0's macroF1: 0.826652
[11]	valid_0's binary_logloss: 0.181639	valid_0's macroF1: 0.837845
[12]	valid_0's binary_logloss: 0.177881	valid_0's macroF1: 0.839369
[13]	valid_0's binary_logloss: 0.174443	valid_0's macroF1: 0.837385
[14]	valid_0's binary_logloss: 0.171348	valid_0's macroF1: 0.840413
[15]	valid_0's binary_logloss: 0.16799	valid_0's macroF1: 0

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



[10]	valid_0's binary_logloss: 0.193344	valid_0's macroF1: 0.804075
[11]	valid_0's binary_logloss: 0.188255	valid_0's macroF1: 0.810014
[12]	valid_0's binary_logloss: 0.184398	valid_0's macroF1: 0.812437
[13]	valid_0's binary_logloss: 0.180773	valid_0's macroF1: 0.80992
[14]	valid_0's binary_logloss: 0.17735	valid_0's macroF1: 0.813939
[15]	valid_0's binary_logloss: 0.174694	valid_0's macroF1: 0.813939
[16]	valid_0's binary_logloss: 0.171739	valid_0's macroF1: 0.815564
[17]	valid_0's binary_logloss: 0.169926	valid_0's macroF1: 0.817085
[18]	valid_0's binary_logloss: 0.167485	valid_0's macroF1: 0.823448
[19]	valid_0's binary_logloss: 0.166202	valid_0's macroF1: 0.831932
[20]	valid_0's binary_logloss: 0.164609	valid_0's macroF1: 0.832602
[21]	valid_0's binary_logloss: 0.163247	valid_0's macroF1: 0.832409
[22]	valid_0's binary_logloss: 0.162132	valid_0's macroF1: 0.830702
[23]	valid_0's binary_logloss: 0.160998	valid_0's macroF1: 0.833524
[24]	valid_0's binary_logloss: 0.160235	valid_0's

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[7]	valid_0's binary_logloss: 0.212485	valid_0's macroF1: 0.771408
[8]	valid_0's binary_logloss: 0.205891	valid_0's macroF1: 0.787239
[9]	valid_0's binary_logloss: 0.200296	valid_0's macroF1: 0.804069
[10]	valid_0's binary_logloss: 0.195181	valid_0's macroF1: 0.809109
[11]	valid_0's binary_logloss: 0.189827	valid_0's macroF1: 0.819008
[12]	valid_0's binary_logloss: 0.185093	valid_0's macroF1: 0.824594
[13]	valid_0's binary_logloss: 0.181413	valid_0's macroF1: 0.82445
[14]	valid_0's binary_logloss: 0.177784	valid_0's macroF1: 0.826748
[15]	valid_0's binary_logloss: 0.17459	valid_0's macroF1: 0.828152
[16]	valid_0's binary_logloss: 0.172302	valid_0's macroF1: 0.833267
[17]	valid_0's binary_logloss: 0.1701	valid_0's macroF1: 0.832409
[18]	valid_0's binary_logloss: 0.168271	valid_0's macroF1: 0.835437
[19]	valid_0's binary_logloss: 0.166474	valid_0's macroF1: 0.83307
[20]	valid_0's binary_logloss: 0.164585	valid_0's macroF1: 0.839935
[21]	valid_0's binary_logloss: 0.163084	valid_0's macroF

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[19]	valid_0's binary_logloss: 0.156036	valid_0's macroF1: 0.842541
[20]	valid_0's binary_logloss: 0.153847	valid_0's macroF1: 0.840173
[21]	valid_0's binary_logloss: 0.152262	valid_0's macroF1: 0.83954
[22]	valid_0's binary_logloss: 0.151059	valid_0's macroF1: 0.83954
[23]	valid_0's binary_logloss: 0.149534	valid_0's macroF1: 0.841669
[24]	valid_0's binary_logloss: 0.14827	valid_0's macroF1: 0.843159
[25]	valid_0's binary_logloss: 0.14723	valid_0's macroF1: 0.843159
[26]	valid_0's binary_logloss: 0.146091	valid_0's macroF1: 0.843159
[27]	valid_0's binary_logloss: 0.145078	valid_0's macroF1: 0.842289
[28]	valid_0's binary_logloss: 0.144214	valid_0's macroF1: 0.843771
[29]	valid_0's binary_logloss: 0.143135	valid_0's macroF1: 0.843512
[30]	valid_0's binary_logloss: 0.142339	valid_0's macroF1: 0.845848
[31]	valid_0's binary_logloss: 0.141409	valid_0's macroF1: 0.847312
[32]	valid_0's binary_logloss: 0.141029	valid_0's macroF1: 0.847899
[33]	valid_0's binary_logloss: 0.140283	valid_0's ma

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[34]	valid_0's binary_logloss: 0.145322	valid_0's macroF1: 0.858278
[35]	valid_0's binary_logloss: 0.144827	valid_0's macroF1: 0.858816
[36]	valid_0's binary_logloss: 0.144382	valid_0's macroF1: 0.861111
[37]	valid_0's binary_logloss: 0.143647	valid_0's macroF1: 0.861636
[38]	valid_0's binary_logloss: 0.143333	valid_0's macroF1: 0.861636
[39]	valid_0's binary_logloss: 0.142935	valid_0's macroF1: 0.860756
[40]	valid_0's binary_logloss: 0.143002	valid_0's macroF1: 0.861636
[41]	valid_0's binary_logloss: 0.142723	valid_0's macroF1: 0.860756
[42]	valid_0's binary_logloss: 0.142204	valid_0's macroF1: 0.861636
[43]	valid_0's binary_logloss: 0.141885	valid_0's macroF1: 0.860756
[44]	valid_0's binary_logloss: 0.141793	valid_0's macroF1: 0.860756
[45]	valid_0's binary_logloss: 0.141348	valid_0's macroF1: 0.860756
[46]	valid_0's binary_logloss: 0.141111	valid_0's macroF1: 0.863038
[47]	valid_0's binary_logloss: 0.141259	valid_0's macroF1: 0.863038
[48]	valid_0's binary_logloss: 0.140747	valid_0'

In [76]:
predicted_y = model.predict(x_test)
train_pred_y = model.predict(x)

In [77]:
print(); print(metrics.classification_report(y,train_pred_y))
print(); print(metrics.confusion_matrix(y, train_pred_y))

print(); print(metrics.classification_report(y_test, predicted_y))
print(); print(metrics.confusion_matrix(y_test, predicted_y))


              precision    recall  f1-score   support

           0       0.95      0.85      0.89      1345
           1       0.98      1.00      0.99     12395

    accuracy                           0.98     13740
   macro avg       0.97      0.92      0.94     13740
weighted avg       0.98      0.98      0.98     13740


[[ 1138   207]
 [   61 12334]]

              precision    recall  f1-score   support

           0       0.87      0.68      0.76       336
           1       0.97      0.99      0.98      3099

    accuracy                           0.96      3435
   macro avg       0.92      0.83      0.87      3435
weighted avg       0.96      0.96      0.96      3435


[[ 228  108]
 [  34 3065]]


<h2>Testing</h2>

In [78]:

test_set = pd.read_csv(data_path+'test.csv', index_col="tripid")


In [79]:
test_set['pickup_time'] = pd.to_datetime(test_set['pickup_time'], format="%m/%d/%Y %H:%M")
test_set['drop_time'] = pd.to_datetime(test_set['drop_time'], format="%m/%d/%Y %H:%M")

In [80]:
test_set = test_set.assign(timeOfDay=pd.cut(test_set.pickup_time.dt.hour,[-1, 8, 20, 24],labels=['dawn','day', 'night']))

In [81]:
test_set.loc[test_set['timeOfDay'] == 'day', 'isNormalCharge'] = 1
test_set.loc[test_set['timeOfDay'] != 'day', 'isNormalCharge'] = 0

In [82]:
durations = []
for index,row in test_set.iterrows():
  provided_duration = row['duration'] #first row of location.lat column here
  if math.isnan(provided_duration) or provided_duration <= 0 :
    time_dif = (row['drop_time'] - row['pickup_time']).seconds
    if(time_dif == 0):
        time_dif = np.nan
    durations.append(time_dif)
  else :  
    durations.append(provided_duration)

test_set.insert(4,"time_dif",durations)

In [83]:
new_column = []                    #empty column for distance
for index,row in test_set.iterrows():
  lat1 = row['pick_lat'] #first row of location.lat column here
  lon1 = row['pick_lon'] #first row of location.long column here
  lat2 = row['drop_lat'] #second row of location.lat column here
  lon2 = row['drop_lon'] #second row of location.long column here
  value = dist_from_coordinates(lat1, lon1, lat2, lon2)  #get the distance
  new_column.append(value)   #append the empty list with distance values

test_set.insert(4,"distance",new_column)

In [84]:
test_set['time_driven'] = test_set['duration']  - test_set['meter_waiting']

In [85]:
chargeperhours = []
for index,row in test_set.iterrows():
    if(row['meter_waiting'] == 0):
        chargeperhour = np.nan
    else:
        chargeperhour = (row['meter_waiting_fare'] / row['meter_waiting'] * 3600)
    chargeperhours.append(chargeperhour)

test_set.insert(4,'charge_per_hour',chargeperhours)


In [86]:
test_set['driving_fare'] = test_set['fare']  - test_set['meter_waiting_fare'] - test_set['additional_fare']

In [87]:
avgspeeds = []
for index,row in test_set.iterrows():
    if(row['time_driven'] == 0):
        avgspeed = np.nan    
    else:
        avgspeed = (row['distance'] / row['time_driven'] * 3600)
    avgspeeds.append(avgspeed)

test_set.insert(4,"avg_speed",avgspeeds)



In [88]:
costsperkm = []
for index,row in test_set.iterrows():
    if row['distance'] == 0:
        costperkm = np.nan
            
    else:
        costperkm = (row['driving_fare'] / row['distance'])
    costsperkm.append(costperkm)

test_set.insert(4,"cost_per_km",costsperkm)

In [89]:
test_features = test_set[training_columns]

In [90]:
test_features.isna().sum()

meter_waiting           0
meter_waiting_fare      0
fare                    0
additional_fare         0
distance                0
cost_per_km            25
avg_speed               9
time_dif                0
time_driven             0
charge_per_hour       298
driving_fare            0
isNormalCharge          0
pick_lat                0
pick_lon                0
drop_lat                0
drop_lon                0
dtype: int64

In [91]:
predicted_labels = model.predict(test_features)

In [92]:
predicted_labels_df = pd.DataFrame(predicted_labels )

In [93]:
sub_path =os.path.abspath(os.path.join(data_path+'/sample_submission.csv'))
submission_set = pd.read_csv(sub_path, index_col="tripid")

In [94]:
submission_set['prediction']= predicted_labels_df.values[:,0]

In [95]:
%%javascript
var kernel = IPython.notebook.kernel;
var thename = window.document.getElementById("notebook_name").innerHTML;
var command = "theNotebook = " + "'"+thename+"'";
kernel.execute(command);

<IPython.core.display.Javascript object>

In [97]:
filename = '../../submissions/'+theNotebook+'/'+theNotebook+'_{%i}.csv'
dirname = '../../submissions/'+theNotebook
fileversion = 1

if not os.path.exists(dirname):
    os.makedirs(dirname)
while glob.glob(filename.replace('{%i}',str(fileversion))) :
    fileversion+=1
submission_set.to_csv(filename.replace('{%i}',str(fileversion)), index=True)
print("Completed!")

Completed!


In [96]:
submission_set['prediction'].value_counts()

1    8117
0     459
Name: prediction, dtype: int64